In [1]:
import os
from os.path import join
import sys

import numpy as np
from nipy.algorithms.registration import HistogramRegistration, Rigid
import nibabel as nib

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# load an example functional image
img = nib.load('tests/GE_func_s1925.nii.gz')

In [3]:
img.get_data().shape

(64, 64, 18, 60)

In [8]:
# create a reference volume and a test volume
affine = img.affine

refVol = img.get_data()[:,:,:,30]
refVol = nib.Nifti1Image(refVol, affine)

testVol = img.get_data()[:,:,:,35]
testVol = nib.Nifti1Image(testVol, affine)

In [19]:
import time

In [34]:
# Create a HR object
reg = HistogramRegistration(testVol, refVol, interp="tri")

# optimize the registration (but you have to give it somewhere to start from)
# In a real-time run, you could do this iteratively, using the previous T as the 
# starting point.
start = time.time()
T = reg.optimize(Rigid(np.eye(4)))
print('took: {}'.format(time.time()-start))

Initial guess...
translation : [ 0.  0.  0.]
rotation    : [ 0.  0.  0.]

Optimizing using fmin_powell
translation : [ 0.01054075 -0.00311031  0.04224973]
rotation    : [  1.72537222e-04   3.23241320e-05  -2.39522826e-05]

crl1 = 0.970112695776

Optimization terminated successfully.
         Current function value: -0.970113
         Iterations: 1
         Function evaluations: 61
took: 0.3690800666809082


In [35]:
T.translation

array([ 0.01054075, -0.00311031,  0.04224973])

In [36]:
T.rotation

array([  1.72537222e-04,   3.23241320e-05,  -2.39522826e-05])

In [37]:
# now that we have an existing alignment transform
# apply to the next volume to see if its faster
testVol2 = img.get_data()[:,:,:,36]
testVol2 = nib.Nifti1Image(testVol2, affine)

In [54]:
reg = HistogramRegistration(testVol2, refVol, from_bins=128, to_bins=128, interp="pv", smooth=10)
start = time.time()
T2 = reg.optimize(T)
print('took: {}'.format(time.time()-start))

Initial guess...
translation : [-0.06089126 -1.30187884  3.57099499]
rotation    : [ -8.38365216e-05  -3.49056598e-05  -3.64426612e-05]

Optimizing using fmin_powell
translation : [-0.06006747 -1.30573526  3.56987801]
rotation    : [ -4.80333827e-05  -3.11382356e-05  -2.46366070e-05]

crl1 = 0.754133006261

Optimization terminated successfully.
         Current function value: -0.754133
         Iterations: 1
         Function evaluations: 80
took: 0.44485926628112793
